In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# CELL 1: Install Required Libraries and Setup GPU
# Run this cell first to install everything you need

import subprocess
import sys

required_packages = [
    'tensorflow[and-cuda]',  # Includes GPU support
    'pandas',
    'numpy',
    'matplotlib',
    'seaborn',
    'scipy',
    'scikit-learn',
    'openpyxl'  # For Excel file reading
]

print("Installing required packages...")
for package in required_packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

print("✅ All packages installed successfully!")
print("\nChecking GPU availability...")
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")
print(f"GPU count: {len(tf.config.list_physical_devices('GPU'))}")

print("\nSetup complete! Ready to proceed.")


Installing required packages...
✅ All packages installed successfully!

Checking GPU availability...
TensorFlow version: 2.10.1
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU count: 1

Setup complete! Ready to proceed.


In [4]:
%pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [5]:
# CELL 2: Import Libraries and Setup Seeds
# All the essential imports in one place

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime
from typing import Dict, List, Tuple, Callable
from scipy import stats
import os
import shutil
from tqdm.notebook import tqdm  # Progress bars for Jupyter!

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input, LSTM, Bidirectional, Dropout,
    BatchNormalization, GlobalMaxPooling1D, Dense
)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("✅ All libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print("Ready for data loading!")


✅ All libraries imported successfully!
TensorFlow version: 2.10.1
NumPy version: 1.23.5
Pandas version: 2.3.3
Ready for data loading!


In [9]:
import os
os.getcwd()
os.listdir()


['.ipynb_checkpoints',
 'data-centres-worldwide.csv',
 'EXCELdata_center_carbon_emission_dataset_75000.csv.xlsx',
 'Frontier HPC & Facility Data.xlsx',
 'global-data-on-sustainable-energy (1).csv',
 'Model1.0NinOA.ipynb',
 'Model1.1NinOA.ipynb',
 'Model1.2NinOA1-Copy1.ipynb',
 'Model1.2NinOA1-Copy2.ipynb',
 'Model1.2NinOA1.ipynb',
 'Untitled.ipynb']

In [10]:
import os

for root, dirs, files in os.walk(os.getcwd()):
    for f in files:
        if "carbon" in f.lower():
            print(os.path.join(root, f))


C:\Users\AnweshaSingh\anaconda_projects\44f72650-e0bb-4a6e-a65c-79ad6ab0e8e1\EXCELdata_center_carbon_emission_dataset_75000.csv.xlsx


In [11]:
# CELL 3: Load Dataset
# Upload your data_center_carbon_emission_dataset_COPY.xlsx file to this folder
# or place it in the same directory as this notebook

print("Loading carbon emission dataset...")
# Adjust the path if your file is in a different location
df = pd.read_excel('anaconda_projects/44f72650-e0bb-4a6e-a65c-79ad6ab0e8e1/EXCELdata_center_carbon_emission_dataset_75000.csv.xlsx')

print(f"✅ Dataset loaded successfully! Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")
print("\nFirst few rows:")
print(df.head())
print("\nColumn names:")
print(df.columns.tolist())


Loading carbon emission dataset...


FileNotFoundError: [Errno 2] No such file or directory: 'anaconda_projects/44f72650-e0bb-4a6e-a65c-79ad6ab0e8e1/EXCELdata_center_carbon_emission_dataset_75000.csv.xlsx'

In [ ]:
# CELL 4: Data Preprocessing with Progress
print("🚀 Starting data preprocessing...")

# Convert timestamp and sort chronologically
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)
print("✓ Timestamp converted and data sorted")

# Forward fill missing values
df.ffill(inplace=True)
print("✓ Missing values forward filled")

# Remove duplicates
original_len = len(df)
df.drop_duplicates(inplace=True)
print(f"✓ Removed {original_len - len(df):,} duplicate rows")

# Create time-based features
df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek
print("✓ Time-based features created")

# Remove outliers using z-score (keeping |z| < 3)
original_len = len(df)
z_scores = np.abs(stats.zscore(df.select_dtypes(include=[np.number])))
df = df[(z_scores < 3).all(axis=1)]
print(f"✓ Removed {original_len - len(df):,} outlier rows")

# Create lag features
df['voltage_lag1'] = df['voltage'].shift(1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print("✓ Lag features created and NaNs dropped")

print(f"\n✅ Final dataset shape: {df.shape}")


In [ ]:
# CELL 5: Train-Validation-Test Split (BEFORE Scaling)
print("📊 Splitting dataset: Train(70%), Val(15%), Test(15%)")

train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)

train = df[:train_size].copy()
val = df[train_size:train_size + val_size].copy()
test = df[train_size + val_size:].copy()

print(f"Train: {train.shape} ({train.shape[0]:,} samples)")
print(f"Val:   {val.shape} ({val.shape[0]:,} samples)")
print(f"Test:  {test.shape} ({test.shape[0]:,} samples)")


In [ ]:
# CELL 6: Feature Scaling and Encoding
print("⚙️ Scaling numerical features...")

num_cols = [
    'voltage', 'current', 'power', 'frequency', 'energy', 'power_factor',
    'esp32_temperature', 'cpu_temperature', 'gpu_temperature',
    'cpu_usage_percent', 'cpu_power_watts', 'gpu_usage_percent',
    'gpu_power_watts', 'ram_usage_percent', 'ram_power_watts'
]

# Fit scaler on ALL data (standard practice for time series)
all_data_temp = pd.concat([train, val, test], ignore_index=True)
scaler = StandardScaler()
all_data_temp[num_cols] = scaler.fit_transform(all_data_temp[num_cols])

# Apply to splits
train[num_cols] = scaler.transform(train[num_cols])
val[num_cols] = scaler.transform(val[num_cols])
test[num_cols] = scaler.transform(test[num_cols])
print("✓ Numerical features scaled")

# One-hot encode categorical variables
train = pd.get_dummies(train, columns=['MAC', 'weekday'], drop_first=True)
val = pd.get_dummies(val, columns=['MAC', 'weekday'], drop_first=True)
test = pd.get_dummies(test, columns=['MAC', 'weekday'], drop_first=True)

# Align columns across splits
all_cols = set(train.columns).union(set(val.columns)).union(set(test.columns))
for col in all_cols:
    if col not in train.columns: train[col] = 0
    if col not in val.columns: val[col] = 0
    if col not in test.columns: test[col] = 0

train, val, test = train[sorted(train.columns)], val[sorted(val.columns)], test[sorted(test.columns)]
print("✓ Columns aligned and encoded")

# Verification
print("\n📈 Energy stats after scaling:")
for split, data in [("Train", train), ("Val", val), ("Test", test)]:
    print(f"  {split:<5} - Mean: {data['energy'].mean():.4f}, Std: {data['energy'].std():.4f}")

print(f"\n✅ Final shapes: Train({train.shape}), Val({val.shape}), Test({test.shape})")


In [ ]:
# CELL 7: Create Sequences for LSTM
print("🔄 Preparing sequences for LSTM...")

def create_sequences(data: np.ndarray, target: np.ndarray, seq_length: int):
    """Create sliding window sequences for time series"""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(target[i + seq_length])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

sequence_length = 10
target_col = 'energy'
feature_cols = [col for col in train.columns if col not in ['timestamp', target_col]]

# Prepare arrays
X_train_arr = train[feature_cols].values
y_train_arr = train[target_col].values
X_val_arr = val[feature_cols].values
y_val_arr = val[target_col].values
X_test_arr = test[feature_cols].values
y_test_arr = test[target_col].values

# Create sequences
X_train_seq, y_train_seq = create_sequences(X_train_arr, y_train_arr, sequence_length)
X_val_seq, y_val_seq = create_sequences(X_val_arr, y_val_arr, sequence_length)
X_test_seq, y_test_seq = create_sequences(X_test_arr, y_test_arr, sequence_length)

print(f"✅ Sequences ready:")
print(f"  Train: {X_train_seq.shape}")
print(f"  Val:   {X_val_seq.shape}")
print(f"  Test:  {X_test_seq.shape}")
print(f"Each sequence: {sequence_length} timesteps × {X_train_seq.shape[2]} features")


In [ ]:
# CELL 8: Scaling Verification
print("=" * 70)
print("🔍 SCALING VERIFICATION")
print("=" * 70)

print("\nTarget (energy) statistics:")
print(f"  Train - Min: {y_train_seq.min():.4f}, Max: {y_train_seq.max():.4f}, Mean: {y_train_seq.mean():.4f}, Std: {y_train_seq.std():.4f}")
print(f"  Val   - Min: {y_val_seq.min():.4f}, Max: {y_val_seq.max():.4f}, Mean: {y_val_seq.mean():.4f}, Std: {y_val_seq.std():.4f}")
print(f"  Test  - Min: {y_test_seq.min():.4f}, Max: {y_test_seq.max():.4f}, Mean: {y_test_seq.mean():.4f}, Std: {y_test_seq.std():.4f}")

print("\n" + "="*70)
print("✅ Expected: Means ≈ 0, Stds ≈ 1 (Scaling successful!)")
print("="*70)


In [ ]:
# CELL 9: Ninja Optimization Algorithm (Simplified for Jupyter)
# Your custom Ninja OA - checkpointing removed, progress bars added

class NinjaOptimizationAlgorithm:
    def __init__(self, objective_function: Callable, bounds: Dict[str, Tuple], 
                 n_agents: int = 12, max_iterations: int = 12, 
                 exploration_factor: float = 2.0, exploitation_factor: float = 0.5,
                 verbose: bool = True):
        self.objective_function = objective_function
        self.bounds = bounds
        self.n_agents = n_agents
        self.max_iterations = max_iterations
        self.exploration_factor = exploration_factor
        self.exploitation_factor = exploitation_factor
        self.verbose = verbose
        
        self.param_names = list(bounds.keys())
        self.agents = self._initialize_population()
        self.fitness = np.full(n_agents, np.inf)
        self.best_agent = None
        self.best_fitness = np.inf
        self.convergence_curve = []
    
    def _initialize_population(self) -> List[Dict]:
        agents = []
        for _ in range(self.n_agents):
            agent = {}
            for param, bound in self.bounds.items():
                if bound[1] == 'int':
                    agent[param] = int(np.random.randint(bound[0][0], bound[0][1] + 1))
                elif bound[1] == 'float':
                    agent[param] = float(np.random.uniform(bound[0][0], bound[0][1]))
                elif bound[1] == 'float_log':
                    log_min, log_max = np.log10(bound[0][0]), np.log10(bound[0][1])
                    agent[param] = float(10 ** np.random.uniform(log_min, log_max))
                elif bound[1] == 'categorical':
                    selected = np.random.choice(bound[0])
                    agent[param] = int(selected) if isinstance(selected, (int, np.integer)) else selected
            agents.append(agent)
        return agents
    
    def _evaluate_fitness(self, agent: Dict) -> float:
        try:
            return self.objective_function(agent)
        except:
            return np.inf
    
    def _exploration_phase(self, agent: Dict, iteration: int) -> Dict:
        new_agent = agent.copy()
        progress = iteration / self.max_iterations
        for param, bound in self.bounds.items():
            exploration_rate = self.exploration_factor * (1 - progress)
            if bound[1] == 'int':
                pert = np.random.randint(-int(exploration_rate), int(exploration_rate) + 1)
                new_agent[param] = int(np.clip(agent[param] + pert, bound[0][0], bound[0][1]))
            elif bound[1] in ['float', 'float_log']:
                range_size = bound[0][1] - bound[0][0]
                pert = np.random.uniform(-exploration_rate, exploration_rate) * range_size * 0.1
                new_agent[param] = float(np.clip(agent[param] + pert, bound[0][0], bound[0][1]))
            elif bound[1] == 'categorical' and np.random.rand() < exploration_rate * 0.3:
                selected = np.random.choice(bound[0])
                new_agent[param] = int(selected) if isinstance(selected, (int, np.integer)) else selected
        return new_agent
    
    def _exploitation_phase(self, agent: Dict, best_agent: Dict, iteration: int) -> Dict:
        new_agent = {}
        progress = iteration / self.max_iterations
        for param, bound in self.bounds.items():
            exploitation_rate = self.exploitation_factor * progress
            if bound[1] == 'int':
                direction = best_agent[param] - agent[param]
                step = int(direction * exploitation_rate)
                new_agent[param] = int(np.clip(agent[param] + step, bound[0][0], bound[0][1]))
            elif bound[1] in ['float', 'float_log']:
                direction = best_agent[param] - agent[param]
                step = direction * exploitation_rate + np.random.normal(0, 0.01 * abs(direction))
                new_agent[param] = float(np.clip(agent[param] + step, bound[0][0], bound[0][1]))
            elif bound[1] == 'categorical':
                new_agent[param] = best_agent[param] if np.random.rand() < 0.7 + 0.3 * progress else agent[param]
        return new_agent
    
    def _adaptive_phase_selection(self, iteration: int) -> str:
        progress = iteration / self.max_iterations
        return 'exploitation' if np.random.rand() < (0.3 + 0.6 * progress) and self.best_agent else 'exploration'
    
    def optimize(self) -> Tuple[Dict, float, List]:
        print("🥷 Ninja Optimization Algorithm starting...")
        print(f"Population: {self.n_agents} agents | Iterations: {self.max_iterations}")
        print(f"Optimizing: {self.param_names}\n")
        
        start_time = time.time()
        
        # Evaluate initial population with progress bar
        print("Evaluating initial population...")
        for i in tqdm(range(self.n_agents), desc="Initial agents"):
            self.fitness[i] = self._evaluate_fitness(self.agents[i])
            if self.fitness[i] < self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_agent = self.agents[i].copy()
        
        self.convergence_curve.append(self.best_fitness)
        print(f"\nInitial best fitness: {self.best_fitness:.6f}\n")
        
        # Main optimization loop with progress bar
        for iteration in tqdm(range(self.max_iterations), desc="Ninja OA"):
            new_agents, new_fitness = [], []
            
            for i, agent in enumerate(self.agents):
                phase = self._adaptive_phase_selection(iteration)
                candidate = (self._exploitation_phase(agent, self.best_agent, iteration) 
                           if phase == 'exploitation' else self._exploration_phase(agent, iteration))
                
                candidate_fitness = self._evaluate_fitness(candidate)
                
                if candidate_fitness < self.fitness[i]:
                    new_agents.append(candidate)
                    new_fitness.append(candidate_fitness)
                else:
                    new_agents.append(agent)
                    new_fitness.append(self.fitness[i])
                
                if candidate_fitness < self.best_fitness:
                    self.best_fitness = candidate_fitness
                    self.best_agent = candidate.copy()
            
            self.agents = new_agents
            self.fitness = np.array(new_fitness)
            self.convergence_curve.append(self.best_fitness)
            
            avg_fitness = np.mean(self.fitness)
            elapsed_h = (time.time() - start_time) / 3600
            if self.verbose:
                print(f"Iter {iteration+1}: Best={self.best_fitness:.6f} | Avg={avg_fitness:.6f} | {elapsed_h:.2f}h")
        
        total_time = (time.time() - start_time) / 3600
        print(f"\n✅ Optimization complete! Total time: {total_time:.2f} hours")
        
        print("\n🥇 Best hyperparameters:")
        for key, value in self.best_agent.items():
            print(f"  {key}: {value}")
        print(f"Best validation loss: {self.best_fitness:.6f}")
        
        return self.best_agent, self.best_fitness, self.convergence_curve

print("✅ Ninja Optimization Algorithm ready!")


In [ ]:
# CELL 10: LSTM Model and Objective Function (Unchanged)
def create_lstm_model(params: Dict, seq_len: int, num_feats: int) -> Sequential:
    model = Sequential(name='LSTM_NiOA')
    model.add(Input(shape=(seq_len, num_feats)))
    
    model.add(Bidirectional(LSTM(params['units'], return_sequences=True)))
    
    for _ in range(params['lstm_layers'] - 1):
        model.add(LSTM(params['units'], return_sequences=True))
        model.add(Dropout(params['dropout']))
    
    model.add(BatchNormalization())
    model.add(GlobalMaxPooling1D())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(params['dropout']))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(1))
    
    optimizer = (tf.keras.optimizers.AdamW(learning_rate=params['learning_rate']) 
                if params['optimizer'] == 'adamw' else 
                tf.keras.optimizers.Adam(learning_rate=params['learning_rate']))
    
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    return model

def objective_function_lstm(params: Dict) -> float:
    model = create_lstm_model(params, X_train_seq.shape[1], X_train_seq.shape[2])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)
    
    history = model.fit(
        X_train_seq, y_train_seq,
        validation_data=(X_val_seq, y_val_seq),
        epochs=30,
        batch_size=params['batch_size'],
        callbacks=[early_stop],
        verbose=0
    )
    
    min_val_loss = min(history.history['val_loss'])
    del model
    tf.keras.backend.clear_session()
    return min_val_loss

print("✅ LSTM model and objective function ready!")
print(f"Global variables set: seq_len={X_train_seq.shape[1]}, num_feats={X_train_seq.shape[2]}")


In [ ]:
# CELL 11: Run Ninja Optimization (The Main Event!)
hyperparameter_bounds = {
    'lstm_layers': ([2, 3], 'int'),
    'units': ([64, 128], 'categorical'),
    'dropout': ([0.3, 0.6], 'float'),
    'optimizer': (['adamw'], 'categorical'),
    'learning_rate': ([5e-5, 5e-4], 'float_log'),
    'batch_size': ([32], 'categorical')
}

print(f"🚀 Starting Ninja OA at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

# Global variables for the optimizer
global seq_len, num_feats
seq_len = X_train_seq.shape[1]
num_feats = X_train_seq.shape[2]

ninja_optimizer = NinjaOptimizationAlgorithm(
    objective_function=objective_function_lstm,
    bounds=hyperparameter_bounds,
    n_agents=12,
    max_iterations=12
)

best_params, best_loss, convergence = ninja_optimizer.optimize()

print("\n" + "="*70)
print("🎉 OPTIMIZATION COMPLETE!")
print("="*70)
print(f"Best validation loss: {best_loss:.6f}")
print(f"Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# CELL 12: Visualize Results
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(convergence, 'b-o', linewidth=3, markersize=8, label='Best Fitness')
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Validation Loss (MSE)', fontsize=12)
plt.title('🥷 Ninja OA Convergence', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(convergence)), convergence, 'go-', linewidth=3, markersize=8)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Best Fitness (log scale)', fontsize=12)
plt.yscale('log')
plt.title('Convergence (Log Scale)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Stats
print("📊 Convergence Statistics:")
print(f"Start: {convergence[0]:.6f}")
print(f"Final: {convergence[-1]:.6f}")
improvement = ((convergence[0] - convergence[-1]) / convergence[0]) * 100
print(f"Improvement: {improvement:.2f}%")

print("\n✅ All done! Your best hyperparameters are ready for final model training.")
print(f"Best params: {best_params}")
